In [ ]:
import tensorflow as tf

In [2]:
dataset = tf.data.Dataset.from_tensor_slices([1,2,3,4,5,6,7])

In [48]:
dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [49]:
for ele in dataset:
    print(ele)

tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)


In [50]:
for ele in dataset:
    print(ele.numpy())#将tensor转化为numpy类型

1
2
3
4
5
6
7


In [51]:
#列表
dataset = tf.data.Dataset.from_tensor_slices([[1,2],[3,4],[5,6]])

In [52]:
dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(2,), dtype=tf.int32, name=None)>

In [53]:
for ele in dataset:
    print(ele.numpy())

[1 2]
[3 4]
[5 6]


In [54]:
#字典
dataset_dic = tf.data.Dataset.from_tensor_slices({
    'a':[1,2,3,4],
    'b':[6,7,8,9],
    'c':[10,23,12,15]
})

In [55]:
dataset_dic

<_TensorSliceDataset element_spec={'a': TensorSpec(shape=(), dtype=tf.int32, name=None), 'b': TensorSpec(shape=(), dtype=tf.int32, name=None), 'c': TensorSpec(shape=(), dtype=tf.int32, name=None)}>

In [56]:
for ele in dataset_dic:
    print(ele)

{'a': <tf.Tensor: shape=(), dtype=int32, numpy=1>, 'b': <tf.Tensor: shape=(), dtype=int32, numpy=6>, 'c': <tf.Tensor: shape=(), dtype=int32, numpy=10>}
{'a': <tf.Tensor: shape=(), dtype=int32, numpy=2>, 'b': <tf.Tensor: shape=(), dtype=int32, numpy=7>, 'c': <tf.Tensor: shape=(), dtype=int32, numpy=23>}
{'a': <tf.Tensor: shape=(), dtype=int32, numpy=3>, 'b': <tf.Tensor: shape=(), dtype=int32, numpy=8>, 'c': <tf.Tensor: shape=(), dtype=int32, numpy=12>}
{'a': <tf.Tensor: shape=(), dtype=int32, numpy=4>, 'b': <tf.Tensor: shape=(), dtype=int32, numpy=9>, 'c': <tf.Tensor: shape=(), dtype=int32, numpy=15>}


In [83]:
import numpy as np

In [84]:
#numpy 列表
dataset = tf.data.Dataset.from_tensor_slices(np.array([1,2,3,4,5,6,7]))

In [59]:
for ele in dataset.take(2):
    print(ele.numpy())

1
2


In [60]:
#迭代器
next(iter(dataset.take(1)))

<tf.Tensor: shape=(), dtype=int64, numpy=1>

In [81]:
dataset = dataset.shuffle(2)#乱续x次
dataset=dataset.repeat(count=3)#循环x次

In [85]:
for ele in dataset:
    print(ele.numpy())

1
2
3
4
5
6
7


In [86]:
dataset=dataset.shuffle(7)#乱续7次
dataset=dataset.repeat(count=3)#重复三次
dataset=dataset.batch(7)#分为3批次

In [87]:
for ele in dataset:
    print(ele.numpy())

[3 1 5 7 2 4 6]
[7 4 5 3 1 6 2]
[6 1 3 5 2 4 7]


In [44]:
dataset = tf.data.Dataset.from_tensor_slices([1,2,3,4,5,6,7])
dataset=dataset.map(tf.square)#对每个数求平方

In [45]:
for ele in dataset:
    print(ele.numpy())

1
4
9
16
25
36
49


## TF.data输入模块实例

In [3]:
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
#加载mnist数字数据集
(train_images,train_labels),(test_images,test_labels) = tf.keras.datasets.mnist.load_data()

In [5]:
#数据归一化操作
train_images=train_images/255.0
test_images=test_images/255.0

---

In [6]:
# #创建tf.data数据
# ds_train_image=tf.data.Dataset.from_tensor_slices(train_images)
# ds_train_lab=tf.data.Dataset.from_tensor_slices(train_labels)

# #合并数据
# ds_train=tf.data.Dataset.zip(ds_train_image,ds_train_lab)

#直接通过元组的方式创建训练数据集
ds_train=tf.data.Dataset.from_tensor_slices((train_images,train_labels))

In [7]:
#data数据有三种处理方法，乱序，重复
ds_train=ds_train.shuffle(10000).repeat().batch(64) #乱序,重复，分批次

---

In [8]:
#将数据放到元组内
ds_test=tf.data.Dataset.from_tensor_slices((test_images,test_labels))

In [9]:
ds_test=ds_test.batch(64)#64批次

---

In [10]:
#创建模型
model=tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(10,activation='softmax')
])

In [16]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy']
             )

In [17]:
#训练步数，60000/64.
#每次循环64次，总共60000图片
steps_per_epochs=train_images.shape[0]/64


In [19]:
model.fit(ds_train,
          epochs=100,
          steps_per_epoch=steps_per_epochs,
          validation_data=ds_test,
          validation_steps=10000/64
         )

Epoch 1/100
937/937 [==============================] - 3s 3ms/step - loss: 0.0169 - accuracy: 0.9955 - val_loss: 0.0741 - val_accuracy: 0.9789
Epoch 2/100
937/937 [==============================] - 3s 3ms/step - loss: 0.0141 - accuracy: 0.9961 - val_loss: 0.0757 - val_accuracy: 0.9792
Epoch 3/100
937/937 [==============================] - 7s 7ms/step - loss: 0.0128 - accuracy: 0.9963 - val_loss: 0.0783 - val_accuracy: 0.9787
Epoch 4/100
937/937 [==============================] - 5s 6ms/step - loss: 0.0091 - accuracy: 0.9978 - val_loss: 0.0777 - val_accuracy: 0.9794
Epoch 5/100
937/937 [==============================] - 4s 4ms/step - loss: 0.0077 - accuracy: 0.9982 - val_loss: 0.0894 - val_accuracy: 0.9755
Epoch 6/100
937/937 [==============================] - 4s 4ms/step - loss: 0.0064 - accuracy: 0.9987 - val_loss: 0.0879 - val_accuracy: 0.9776
Epoch 7/100
937/937 [==============================] - 4s 4ms/step - loss: 0.0079 - accuracy: 0.9978 - val_loss: 0.0920 - val_accuracy: 0.9767

937/937 [==============================] - 4s 4ms/step - loss: 0.0025 - accuracy: 0.9992 - val_loss: 0.1401 - val_accuracy: 0.9791
Epoch 58/100
937/937 [==============================] - 4s 4ms/step - loss: 0.0025 - accuracy: 0.9992 - val_loss: 0.1356 - val_accuracy: 0.9786
Epoch 59/100
937/937 [==============================] - 5s 5ms/step - loss: 0.0027 - accuracy: 0.9991 - val_loss: 0.1390 - val_accuracy: 0.9815
Epoch 60/100
937/937 [==============================] - 4s 4ms/step - loss: 4.0506e-04 - accuracy: 0.9999 - val_loss: 0.1261 - val_accuracy: 0.9813
Epoch 61/100
937/937 [==============================] - 4s 4ms/step - loss: 9.1263e-05 - accuracy: 1.0000 - val_loss: 0.1278 - val_accuracy: 0.9816
Epoch 62/100
937/937 [==============================] - 4s 5ms/step - loss: 6.8238e-05 - accuracy: 1.0000 - val_loss: 0.1363 - val_accuracy: 0.9810
Epoch 63/100
937/937 [==============================] - 4s 5ms/step - loss: 0.0058 - accuracy: 0.9981 - val_loss: 0.1389 - val_accuracy: 